<a href="https://colab.research.google.com/github/agalvezm/ACE2_scRNAseq/blob/master/filter_concatenate_anndatas/filter_concatenate_anndatas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Pre-processing of anndata

In [13]:
!pip install scikit-misc

     |████████████████████████████████| 8.7MB 3.9MB/s 


In [14]:
from skmisc.loess import loess

In [39]:
!pip install scvi-tools==0.8.0b0


In [1]:
!pip install scikit-misc
!pip install pillow scikit-learn

# Install scvi
#!pip install scvi-tools
!pip install scvi-tools==0.8.0b0


!pip install scanpy anndata

!pip install scanpy python-igraph louvain MulticoreTSNE pybiomart

#import packages
import scanpy as sc
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import anndata
import scvi


# Import packages
import anndata
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scanpy as sc
from sklearn.decomposition import TruncatedSVD
from scipy import sparse, io

matplotlib.rcParams.update({'font.size': 12})
%config InlineBackend.figure_format = 'retina'

def nd(arr):
    return np.asarray(arr).reshape(-1)

We first need to transfer the metadata from the uns dictionary to obs columns. Will need a different notebook to do this but I can start drafting the code here.

## Download the data from Box

In [2]:
!wget -O results.zip https://caltech.box.com/shared/static/0ddcz5qx0xnfxp77gerliu53re5u9p1m

--2021-01-27 00:23:39--  https://caltech.box.com/shared/static/0ddcz5qx0xnfxp77gerliu53re5u9p1m
Resolving caltech.box.com (caltech.box.com)... 107.152.24.197
Connecting to caltech.box.com (caltech.box.com)|107.152.24.197|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /public/static/0ddcz5qx0xnfxp77gerliu53re5u9p1m [following]
--2021-01-27 00:23:39--  https://caltech.box.com/public/static/0ddcz5qx0xnfxp77gerliu53re5u9p1m
Reusing existing connection to caltech.box.com:443.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://caltech.app.box.com/public/static/0ddcz5qx0xnfxp77gerliu53re5u9p1m [following]
--2021-01-27 00:23:39--  https://caltech.app.box.com/public/static/0ddcz5qx0xnfxp77gerliu53re5u9p1m
Resolving caltech.app.box.com (caltech.app.box.com)... 107.152.24.201
Connecting to caltech.app.box.com (caltech.app.box.com)|107.152.24.201|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: h

In [3]:
!unzip results.zip

Archive:  results.zip
   creating: results/
  inflating: __MACOSX/._results      
  inflating: results/resultGSM3319036.dms  
  inflating: __MACOSX/results/._resultGSM3319036.dms  
  inflating: results/resultGSM2803967.dms  
  inflating: __MACOSX/results/._resultGSM2803967.dms  
  inflating: results/resultGSM3509839  
  inflating: __MACOSX/results/._resultGSM3509839  
  inflating: results/resultGSM2905292.dms  
  inflating: __MACOSX/results/._resultGSM2905292.dms  
  inflating: results/resultGSM3439917  
  inflating: __MACOSX/results/._resultGSM3439917  
  inflating: results/resultGSM2905293.dms  
  inflating: __MACOSX/results/._resultGSM2905293.dms  
  inflating: results/resultGSM3439921  
  inflating: __MACOSX/results/._resultGSM3439921  
  inflating: results/resultGSM3439926  
  inflating: __MACOSX/results/._resultGSM3439926  
  inflating: results/resultGSM3439919  
  inflating: __MACOSX/results/._resultGSM3439919  
  inflating: results/resultGSM3319037.dms  
  inflating: __MACOSX/r

First get list of names of anndata objects

In [2]:
import os
adata_names = os.listdir("/content/results")

In [15]:
invalid_names = [name for name in adata_names if ("result" not in name and "SRR" not in name) or "fastq" in name]

In [16]:
invalid_names

['SRR5481335_1.fastq.gz']

In [17]:
for invalid_name in invalid_names:
  adata_names.remove(invalid_name)

In [18]:
adata_objects = {}
wrong_adatas = []


In [19]:
for adata in adata_names:
  # Read adata into dictionary
  print("Starting " + adata)

  adata_objects[adata] = anndata.read_h5ad("/content/results/" + adata);

  adata_objects[adata].var_names_make_unique()

  # transfer metadata from uns to obs columns
  for key, value in adata_objects[adata].uns.items():
    adata_objects[adata].obs[key] = value

  # Reduce dimensionality
  try:
    sc.pp.highly_variable_genes(
      adata_objects[adata],
      n_top_genes=3000,
      subset=True,
      #layer="counts", in the tutorial, they make a layer for raw counts and another for normalized. we won't do this so think this is not necessary.
      flavor="seurat_v3"
    )
  except:
    wrong_adatas.append(adata)
  finally:
    pass
  adata_objects[adata]
  print("Finishing " + adata)



Variable names are not unique. To make them unique, call `.var_names_make_unique`.


Starting resultGSM3439916.dms


/usr/local/lib/python3.6/dist-packages/scanpy/preprocessing/_highly_variable_genes.py:144: FutureWarning: Slicing a positional slice with .loc is not supported, and will raise TypeError in a future version.  Use .loc with labels or .iloc with positions instead.
  df.loc[: int(n_top_genes), 'highly_variable'] = True
Variable names are not unique. To make them unique, call `.var_names_make_unique`.


Finishing resultGSM3439916.dms
Starting resultGSM3319041.dms


/usr/local/lib/python3.6/dist-packages/scanpy/preprocessing/_highly_variable_genes.py:144: FutureWarning: Slicing a positional slice with .loc is not supported, and will raise TypeError in a future version.  Use .loc with labels or .iloc with positions instead.
  df.loc[: int(n_top_genes), 'highly_variable'] = True
Variable names are not unique. To make them unique, call `.var_names_make_unique`.


...

/usr/local/lib/python3.6/dist-packages/scanpy/preprocessing/_highly_variable_genes.py:144: FutureWarning: Slicing a positional slice with .loc is not supported, and will raise TypeError in a future version.  Use .loc with labels or .iloc with positions instead.
  df.loc[: int(n_top_genes), 'highly_variable'] = True


Finishing resultGSM3738536
Starting resultGSM2928382.dms


Variable names are not unique. To make them unique, call `.var_names_make_unique`.


Finishing resultGSM2928382.dms


/usr/local/lib/python3.6/dist-packages/scanpy/preprocessing/_highly_variable_genes.py:144: FutureWarning: Slicing a positional slice with .loc is not supported, and will raise TypeError in a future version.  Use .loc with labels or .iloc with positions instead.
  df.loc[: int(n_top_genes), 'highly_variable'] = True


In [50]:
wrong_adatas

['resultGSM3319047.dms']

In [20]:
adata_objects

{'resultGSM2786157.dms': AnnData object with n_obs × n_vars = 1156 × 3000
     obs: 'cell_counts', 'n_genes', 'percent_mito', 'pass_count_filter', 'pass_mito_filter', 'cell_type', 'condition', 'database_id', 'figure', 'paper', 'sample_id', 'species', 'technology', 'tissue'
     var: 'gene_id', 'gene_name', 'gene_counts', 'n_cells', 'pass_gene_filter', 'highly_variable', 'highly_variable_rank', 'means', 'variances', 'variances_norm'
     uns: 'cell_type', 'condition', 'database_id', 'figure', 'paper', 'sample_id', 'species', 'technology', 'tissue', 'hvg',
 'resultGSM2786158.dms': AnnData object with n_obs × n_vars = 919 × 3000
     obs: 'cell_counts', 'n_genes', 'percent_mito', 'pass_count_filter', 'pass_mito_filter', 'cell_type', 'condition', 'database_id', 'figure', 'paper', 'sample_id', 'species', 'technology', 'tissue'
     var: 'gene_id', 'gene_name', 'gene_counts', 'n_cells', 'pass_gene_filter', 'highly_variable', 'highly_variable_rank', 'means', 'variances', 'variances_norm'
    

In [43]:
# Turns out we have duplicated names from (did with SRA table) Until I figure this out I will just rename then:

def uniq(input):
  a = 0
  output = []
  for x in input:
    if x not in output:
      output.append(x)
    else:
      output.append(x + str(a))
      a+=1
  return output


adata_keys_unique = uniq(adata_keys)

In [21]:
# Get list of adata to concatenate
adata_list = [adata_objects[name] for name in adata_names]
adata_keys = [nd(adata_objects[name].obs["sample_id"])[0] for name in adata_names]

In [44]:
concat_samples = anndata.concat(adata_list, join='outer', label = "sample_id", keys = adata_keys_unique, index_unique="_")

In [48]:
np.unique(concat_samples.obs["tissue"])

array(['adipose', 'bladder', 'blood', 'bone marrow', 'brain',
       'brain organoid', 'eye', 'kidney', 'lung', 'mammary',
       'olfactory epithelium', 'ovary', 'peritoneal', 'prostate',
       'respiratory epithelium', 'skin', 'testis', 'tonsil'], dtype=object)

In [49]:
concat_samples.write("concatenated_adatas_3000genes.dms")

... storing 'cell_type' as categorical
... storing 'condition' as categorical
... storing 'database_id' as categorical
... storing 'figure' as categorical
... storing 'paper' as categorical
... storing 'species' as categorical
... storing 'technology' as categorical
... storing 'tissue' as categorical
